# Data preprocessing

In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
file_path = "/home/yixin/work/msxf/CQU_TimeSeries_Algo/MultiPatch_super/dataset/"
# file_name = "train_dev_8.csv"
file_name = "train_dev1.csv"
df = pd.read_csv(file_path+file_name)
df.head()

,pool_id,timestamp,cpu_usage,cpu_allocated,mem_use,mem_allocated
0,1,1701784800,7.35,89.23,669391777792,1986046564774
1,1,1701788400,7.28,89.23,675155816448,1986046564774
2,1,1701792000,7.25,89.23,664276680704,1986046564774
3,1,1701795600,7.18,89.23,665704325120,1986046564774
4,1,1701799200,7.17,89.23,665549680640,1986046564774


In [3]:
df['pool_id'].value_counts()

pool_id
8     5510
17    5509
1     5508
16    5506
4     5504
13    5504
12    5503
9     5500
14    5498
18    4724
20    4568
21    4567
19    4567
23    4188
24    4110
26    3482
29    3162
3     2166
31    1355
33    1258
35     465
32      81
25       4
Name: count, dtype: int64

In [4]:
nunique_res = df.groupby(by='pool_id').agg(lambda x: x.nunique())
nunique_res

,timestamp,cpu_usage,cpu_allocated,mem_use,mem_allocated
pool_id,,,,,
1,5508,1432,1430,5416,2081
3,2166,1,1,1,1
4,5504,215,77,5438,68
8,5510,917,980,5167,1060
9,5500,992,190,5046,191
12,5503,711,314,5384,331
13,5504,265,81,5232,89
14,5498,2660,35,5473,39
16,5506,229,242,5286,253


In [5]:
ncount_res = df.groupby(by='pool_id').count()
ncount_res

,timestamp,cpu_usage,cpu_allocated,mem_use,mem_allocated
pool_id,,,,,
1,5508,5508,5508,5508,5508
3,2166,2166,2166,2166,2166
4,5504,5504,5504,5504,5504
8,5510,5510,5510,5510,5510
9,5500,5500,5500,5500,5500
12,5503,5503,5503,5503,5503
13,5504,5504,5504,5504,5504
14,5498,5498,5498,5498,5498
16,5506,5506,5506,5506,5506


In [7]:
zero_count = df.groupby(by='pool_id').agg(lambda x: sum(x==0)/len(x))
zero_count

,timestamp,cpu_usage,cpu_allocated,mem_use,mem_allocated
pool_id,,,,,
1,0.0,0.000726,0.000182,0.000726,0.000182
3,0.0,1.000000,1.000000,1.000000,1.000000
4,0.0,0.000727,0.000182,0.000727,0.000182
8,0.0,0.062432,0.061706,0.061343,0.061525
9,0.0,0.063455,0.062364,0.062545,0.061636
12,0.0,0.000182,0.000182,0.000182,0.000182
13,0.0,0.000000,0.000000,0.000000,0.000000
14,0.0,0.000000,0.000000,0.000000,0.000000
16,0.0,0.000000,0.000000,0.000000,0.000000


In [10]:

pool_ids_nunique = (nunique_res.loc[nunique_res["cpu_usage"]>=200,:].reset_index())['pool_id'].unique().tolist()
pool_ids_ncount = (ncount_res.loc[ncount_res['cpu_usage']>=2000, :].reset_index())['pool_id'].unique().tolist()
pool_ids_non_zeros = (zero_count.loc[zero_count["cpu_usage"]<=0.5,:].reset_index())['pool_id'].unique().tolist()
valid_pool_id = set(pool_ids_ncount).intersection(set(pool_ids_nunique)).intersection(set(pool_ids_non_zeros))
valid_pool_id

{1, 4, 8, 9, 12, 13, 14, 16, 20, 23, 24, 26}

In [11]:
out_df = df[df['pool_id'].isin(valid_pool_id)]#to_csv("/home/yixin/work/msxf/CQU_TimeSeries_Algo/MultiPatch_super/dataset/filtered_dev.csv", index=False)
out_df = out_df.sort_values(by=['pool_id',"timestamp"]).reset_index(drop=True)
out_df.head()

,pool_id,timestamp,cpu_usage,cpu_allocated,mem_use,mem_allocated
0,1,1701784800,7.35,89.23,669391777792,1986046564774
1,1,1701788400,7.28,89.23,675155816448,1986046564774
2,1,1701792000,7.25,89.23,664276680704,1986046564774
3,1,1701795600,7.18,89.23,665704325120,1986046564774
4,1,1701799200,7.17,89.23,665549680640,1986046564774


In [13]:
out_df['pool_id'].value_counts()

pool_id
8     5510
1     5508
16    5506
4     5504
13    5504
12    5503
9     5500
14    5498
20    4568
23    4188
24    4110
26    3482
Name: count, dtype: int64

In [101]:
# out_df.to_csv("/home/yixin/work/msxf/CQU_TimeSeries_Algo/MultiPatch_super/dataset/filtered_dev.csv", index=False)

# Dataset Generation

### One for all

### One for one

In [15]:
output_dir = "/home/yixin/work/msxf/CQU_TimeSeries_Algo/MultiPatch_super/dataset/"
for pool_id in out_df['pool_id'].unique().tolist():
    pool_data = out_df.loc[out_df['pool_id']==pool_id,:]
    pool_data = pool_data.sort_values(by=['timestamp']).reset_index(drop=True)
    pool_data.to_csv(output_dir + 'one_for_one_pool_' +str(pool_id) +".csv", index=False)

# Result Analyze

In [15]:
import re
import numpy as np

result = """

336_192_Mamba_one_for_one_pool_8_ftM_sl336_ll48_pl192_dm16_nh4_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_0  
mse:3.573953151702881, mae:1.4876165390014648, rse:1.0441125631332397

336_192_Mamba_one_for_one_pool_8_ftM_sl336_ll48_pl192_dm16_nh4_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_1  
mse:3.677433490753174, mae:1.4470375776290894, rse:1.0591202974319458

336_192_Mamba_one_for_one_pool_8_ftM_sl336_ll48_pl192_dm16_nh4_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_2  
mse:3.5533807277679443, mae:1.4888774156570435, rse:1.0411031246185303

336_192_Mamba_one_for_one_pool_8_ftM_sl336_ll48_pl192_dm16_nh4_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_3  
mse:3.7699334621429443, mae:1.457558274269104, rse:1.0723577737808228

336_192_Mamba_one_for_one_pool_8_ftM_sl336_ll48_pl192_dm16_nh4_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_4  
mse:3.826662063598633, mae:1.4379796981811523, rse:1.0803958177566528

"""

mse = re.findall(".*mse:(.*?),", result)
mae = re.findall(".*mae:(.*?),", result)
rse = re.findall(".*rse:(.*?)\n", result)

mse = [float(x) for x in mse]
mae = [float(x) for x in mae]
rse = [float(x) for x in rse]

med_mse = np.round(np.median(mse),3)
med_mae = np.round(np.median(mae),3)
med_rse = np.round(np.median(rse),3)

avg_mse = np.round(np.mean(mse),3)
avg_mae = np.round(np.mean(mae),3)
avg_rse = np.round(np.mean(rse),3)

print(avg_mse,f"({med_mse})",'\t', avg_mae,f'({med_mae})', '\t',avg_rse, f'({med_rse})')

2.416 (2.425) 	 0.956 (0.958) 	 1.048 (1.05)
